In which I attempt to control multiple robots simultaneously!

In [1]:
# Foundational Sphero API, forked from Simon's Library
import sphero

In [2]:
# Tracking code
import cv2
# SpheroNav base classes
from tracker.trackingfilter import FilterSpheroBlueCover, FilterSpheroYellowCover, FilterGlow


from tracker.traceable import TraceableObject
from tracker.trackerbase import ColorTracker
from util import Vector2D

In [3]:
# My first python module!
import SpheroTeam

from SpheroTeam.teamutil import readJsonFile, normalize_angle, angle_between_points
from SpheroTeam.navigation import calibrate_bot_direction, get_bot_position
from SpheroTeam.formations import roll_polygon

In [62]:
# General Python Imports
import time
import os
import logging

#### SpheroTeam Setup


For more details about what each line does, refer to the previous notebook (6)


In [7]:
# This is a list of the colored "initial names" of the spheros you want to work with in this session
ACTIVEBOTS = ['GRY', 'YPR']

In [ ]:
PROJ_ROOT = os.getcwd()
CONFIG = os.path.join(PROJ_ROOT, "config.json")
manager = sphero.SpheroManager()

# Initialize Sphero manager using options provided in the config file
manager= SpheroTeam.initialize.load_sphero_roster(manager, CONFIG)

# Connect to robots
bots = SpheroTeam.initialize.connect_sphero_team(manager, ACTIVEBOTS)

# Check power, robots shut down after dropping below 7 volts
SpheroTeam.print_team_status(bots)

#### Camera Tracker Setup



In which I try custom filters for yellow, red, green, and orange, find that the camera has trouble seeing ANY of them besides blue


In [9]:
tracker = ColorTracker()

In [31]:
imageX, imageY = tracker.image_size
print "Image: {} x {} pixels".format(imageX, imageY)

Image: 640 x 480 pixels


In [10]:
TRACEABLE_COLORS = [
    [255, 255, 0], # Y
    [0, 0, 255]    # B
]

traceable_blue = TraceableObject("BLUE")
traceable_blue.filter = FilterSpheroBlueCover()

traceable_yellow = TraceableObject("YELLOW")
traceable_yellow.filter = FilterSpheroYellowCover()

In [11]:
# Team Calibration Routine
def get_team_offsets(bots, traceable_object, traceable_color):
    offsets = []
    
    for bot in bots:
        offset = calibrate_bot_direction(bot,  traceable_object, traceable_color, tracker, True)
        offsets.append(offset)
        proceed = raw_input("'q' to quit, else calibrate next robot ")
        if proceed == "q":
            break
        else:
            continue
            
    return offsets

In [12]:
# Didn't seem to work
# offsets = get_team_offsets(bots, traceable_yellow, TRACEABLE_COLORS[0])

Start (226,185)
End   (440,162)
Angle 353.865590056
'q' to quit, else continued
Start (125,336)
End   (423,151)
Angle 328.167767608
'q' to quit, else continued


In [15]:
# Blue seems to be the most reliable color to track with.
offsets = get_team_offsets(bots, traceable_blue, TRACEABLE_COLORS[1])

Start (393,71)
End   (411,45)
Angle 304.695153531
'q' to quit, else continuea
Start (262,181)
End   (295,209)
Angle 40.3141001605
'q' to quit, else continuea


In [11]:
?roll_polygon

In [24]:
SpheroTeam.roll_sphero(bots[0], 80, 180, offsets[0])

In [16]:
# Roll in a square to show it's working
roll_polygon(bots, 1, offsets, 0, speed=60)

Dispatch Time 0.0490000247955


### Lastly: Try for Basic P Controller

Only using p controller on velocity


In [39]:
import math

In [144]:
def bot_to_point(bot, offset,
                 targetX, targetY, 
                 trace_object, trace_color,
                 TIMEOUT=800,
                 tracker=tracker, MAX_SECONDS=10, stopRadius=30,
                MAX_X=imageX, MAX_Y=imageY, Kp=0.25, DEBUG=False):
    
    """
        Currently makes use of a few implicit globals for simplicity
        
        specifically traceable_blue, traceable_color
        Maybe set stop radius  based on pixel size
        
        Wiggles because aiming for direct path / precision
        
        Constrast to "RUSH TO POINT IN STRAIGHT LINE"
        
        Really would work better with a bigger field
        
        Maybe we can push lighter things?
        
        Or boost speed when within range: pause and then burst
    """
    
    if (targetX > MAX_X) or ( targetY > MAX_Y):
        logging.warning("Can't roll off the screen!")
        return -1
    
    currentX, currentY = get_bot_position(bot, trace_object, tracker, samples=1)

    # Basic closed loop controller
    startTime = time.time()
    
    # Angle to distance
    angle, distance = distance_to_heading(currentX, currentY, targetX, targetY)
    
    print("Get from {},{} to {},{}| Distance {} / {}").format(currentX, currentY, 
                                                         targetX, targetY, distance, angle)
    bot.set_motion_timeout(TIMEOUT)
    # REPLACE SOMEDAY WITH TRUE PID CONTROLLER
    while distance > stopRadius and (((time.time() - startTime ) < MAX_SECONDS)):
        outSpeed = distance * Kp + 10 # Constant of proportional control
        
        if outSpeed < 30:
            outSpeed = 35

        # roll the sphero, make use of the request object
        if DEBUG:
            print("Dist {} outSpeed {} at {} degrees: {},{}"\
                  .format(distance, outSpeed, angle, currentX, currentY))
        
        SpheroTeam.roll_sphero(bot, outSpeed, -angle, offset)
        
        # use this for recovery when lost
        
        bot.prev_angle = -angle
                
        time.sleep(TIMEOUT/1000.0)

        currentX , currentY = get_bot_position(bot, trace_object, tracker, samples=1)
        
        if currentX:
            # Repeat waypointing calculation
            angle, distance = distance_to_heading(currentX, currentY, targetX, targetY)
        else:
            # bring robot back onto screen
            angle = bot_prev_angle
            distance = 250 # aim to go back to middle
            
    print("Stopped at {},{}, with dist {}").format(currentX, currentY, distance)
    

def distance_to_heading(currentX, currentY, targetX, targetY):
    '''
        Returns distance and angle between two points
    '''
    
    deltaX = targetX - currentX
    deltaY = targetY - currentY
    angle = math.degrees(math.atan2(deltaY, deltaX))
    distance = math.sqrt(deltaX * deltaX + deltaY * deltaY)
    
    return angle, distance

In [150]:
%%time
# Cut down on sampling to reduce tracking latency
currentX , currentY = get_bot_position(bots[0], traceable_blue, tracker, samples=1)

Wall time: 5 ms


In [151]:
currentX , currentY

(518, 313)

Received unknown async msg! Header:  (255, 254, 5, 1)


Exception in thread SpheroReceiverThread:
Traceback (most recent call last):
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 801, in __bootstrap_inner
    self.run()
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "C:\Users\cyick\AppData\Roaming\Python\Python27\site-packages\sphero\core.py", line 422, in _receiver
    header = self._receive_header()
  File "C:\Users\cyick\AppData\Roaming\Python\Python27\site-packages\sphero\core.py", line 306, in _receive_header
    first_byte = struct.unpack('B', raw_data)[0]
error: unpack requires a string argument of length 1



In [147]:
bots[0].set_rgb(0, 0, 255)

One problem: when robot rolls off the viewing field. Maybe when a robot cannot be seen, it should roll in the opposite of the last angle it rolled in.


In [148]:
bot_to_point(bots[0], offsets[0], 50, 100,
            traceable_blue, TRACEABLE_COLORS[1],
             TIMEOUT=100)

Get from 45,100 to 50,100| Distance 5.0 / 0.0
Stopped at 45,100, with dist 5.0


In [149]:
bot_to_point(bots[0], offsets[0], 500, 300,
            traceable_blue, TRACEABLE_COLORS[1],
             
             TIMEOUT=250)


Get from 44,100 to 500,300| Distance 497.931722227 / 23.6820877245
Stopped at 493,308, with dist 10.6301458127


In [114]:
# I think the key is small timeouts
bot_to_point(bots[0], offsets[0], 150, 150, 
             traceable_blue, TRACEABLE_COLORS[1],
             TIMEOUT=400)

bot_to_point(bots[0], offsets[0], 
             250, 400,
             traceable_blue, TRACEABLE_COLORS[1],
             TIMEOUT=400)

Get from 301,307 to 150,150| Distance 217.830209108 / -133.883989609
Dist 217.830209108 outSpeed 54.457552277 at -133.883989609 degrees: 301,307
Dist 207.932681414 outSpeed 51.9831703535 at -136.169139328 degrees: 300,294
Dist 206.041258004 outSpeed 51.5103145011 at -137.950779109 degrees: 303,288
Dist 179.655225362 outSpeed 44.9138063406 at -136.353191954 degrees: 280,274
Dist 136.124942608 outSpeed 34.031235652 at -139.170436525 degrees: 253,239
Dist 96.5401470892 outSpeed 24.1350367723 at -140.042451069 degrees: 224,212
Dist 58.412327466 outSpeed 14.6030818665 at -141.952957468 degrees: 196,186
Stopped at 165,160, with dist 18.0277563773
Get from 145,141 to 250,400| Distance 279.474506887 / 67.9321004376
Dist 279.474506887 outSpeed 69.8686267219 at 67.9321004376 degrees: 145,141
Dist 281.538629676 outSpeed 70.3846574191 at 67.4427533653 degrees: 142,140
Dist 305.565050358 outSpeed 76.3912625894 at 65.0284175283 degrees: 121,123
Dist 263.518500299 outSpeed 65.8796250748 at 65.0881880

Wow! Robots used to be inching forwards like inchworms, and now the roll smoothly into position!! Although if target is near edge they may overshoot.

Post-script: the improvement in this notebook was mainly small tuning to sampling rate and refinement of the underlying location tracker function in order to improve the tightness of the feedback loop. An actual coordinated pushing scheme will be tried in the next notebook.

